In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col, sum, avg, count, first, last, min, max
spark = SparkSession.builder.appName("sparkApp").getOrCreate()

25/02/10 14:35:43 WARN Utils: Your hostname, apples-MacBook-Air-5.local resolves to a loopback address: 127.0.0.1; using 192.168.1.129 instead (on interface en0)
25/02/10 14:35:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/10 14:35:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/02/10 14:35:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/02/10 14:35:45 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


25/02/10 14:35:59 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


### Spark Joins

In [2]:
log_data1 = spark.read.csv('Data/log_data1.csv', header=True, inferSchema=True)
log_data2 = spark.read.csv('Data/log_data2.csv', header=True, inferSchema=True)
log_data3 = spark.read.csv('Data/log_data3.csv', header=True, inferSchema=True)

In [3]:
log_data1.show(5)

+----------+--------------------+-------------+--------------+-------+
|      date|             user_id|   event_name|mobile_os_type|version|
+----------+--------------------+-------------+--------------+-------+
|12/30/2019|6bcbabd60e6a2e43a...|push_received|        U683CL| 11.3.1|
|12/31/2019|313e76b3dbb52b6ce...|push_received|        U683CL| 11.3.1|
|12/31/2019|69a8b09b5672fcc5f...|push_received|        U683CL| 11.3.1|
|12/31/2019|68413224fbfb9aa33...|push_received|        U683CL| 11.3.1|
|12/30/2019|a79f93465f558f433...|push_received|        U683CL| 11.3.1|
+----------+--------------------+-------------+--------------+-------+
only showing top 5 rows



In [4]:
log_data3.show(5)

+--------+--------------------+-------------+--------------+-------+
|    date|             user_id|   event_name|mobile_os_type|version|
+--------+--------------------+-------------+--------------+-------+
|1/3/2020|325b61cf1c4dc2a51...|push_received|        U683CL| 11.3.0|
|1/4/2020|ad32a2073e48423c2...|  screen_view|        U683CL| 11.3.1|
|1/5/2020|05663e0cfd5550ea2...|push_received|        U683CL| 11.3.1|
|1/6/2020|8d2f1bc552b468bc1...|push_received|        U683CL| 11.3.1|
|1/7/2020|d5474118fa0900557...|push_received|           L51| 11.3.1|
+--------+--------------------+-------------+--------------+-------+
only showing top 5 rows



In [5]:
log_data2.show(5)

+--------------------+--------------------+--------------+------------+
|             user_id|         description|       country|items_bought|
+--------------------+--------------------+--------------+------------+
|6bcbabd60e6a2e43a...|User logged into app|United Kingdom|   HP Laptop|
|313e76b3dbb52b6ce...|User logged into app|United Kingdom|   HP Laptop|
|69a8b09b5672fcc5f...|User logged into app|United Kingdom|   HP Laptop|
|68413224fbfb9aa33...|User logged into app|United Kingdom|   HP Laptop|
|a79f93465f558f433...|User logged into app|United Kingdom|   HP Laptop|
+--------------------+--------------------+--------------+------------+
only showing top 5 rows



# Dataframe join syntax
df1.join(df2, joinExpression, joinType).show()

In [7]:
# Let's perform an inner join
log_data1.join(log_data2, log_data1.user_id==log_data2.user_id, 'inner')\
.select(log_data1.user_id, 'event_name', 'description', 'items_bought').show(5)

+--------------------+-------------+--------------------+------------+
|             user_id|   event_name|         description|items_bought|
+--------------------+-------------+--------------------+------------+
|6bcbabd60e6a2e43a...|push_received|User logged into app|   HP Laptop|
|313e76b3dbb52b6ce...|push_received|User logged into app|   HP Laptop|
|69a8b09b5672fcc5f...|push_received|User logged into app|   HP Laptop|
|68413224fbfb9aa33...|push_received|User logged into app|   HP Laptop|
|a79f93465f558f433...|push_received|User logged into app|   HP Laptop|
+--------------------+-------------+--------------------+------------+
only showing top 5 rows



In [8]:
# Let's perform a left join where item is null
log_data1.join(log_data2, log_data1.user_id==log_data2.user_id, 'left')\
.select(log_data1.user_id, 'event_name', 'description', 'items_bought')\
.where('items_bought IS NULL').show()

+--------------------+-------------+-----------+------------+
|             user_id|   event_name|description|items_bought|
+--------------------+-------------+-----------+------------+
|2hhd72fcc3827b456...|   model_view|       NULL|        NULL|
|h343aed942faabba1...|push_received|       NULL|        NULL|
|456ce1917f02b0051...| push_clicked|       NULL|        NULL|
|7823aed942faabba1...|push_received|       NULL|        NULL|
|54r1e30403f2ef3c1...|push_received|       NULL|        NULL|
|fh5cbd8c37550ba73...|push_received|       NULL|        NULL|
|523641efcb0686e0c...|  screen_view|       NULL|        NULL|
+--------------------+-------------+-----------+------------+



In [9]:
# Combine dataframe using Union
# This allows for duplicate
log_with_dup = log_data1.union(log_data3)
log_with_dup.count()

1003

In [10]:
# Combine dataframe using Union
# This allows for without duplicate
log_without_dup = log_data1.union(log_data3).distinct()
log_without_dup.count()

1001

### Spark SQL 

In [13]:
# To use the Spark SQL API we need to first create a View/Temporary table from a Dataframe
# Let's read Employees data as a view
spark.read\
.csv('Data/employees.csv', header=True, inferSchema=True)\
.createOrReplaceTempView('employee_view')

In [14]:
# Let's query our view using Spark SQL
# We use the Spark SQL method of the SparkSession
employee_view = spark.sql('select * from employee_view')
employee_view.show()

+---------+------------+------+
|     Name| Departments|salary|
+---------+------------+------+
|    Krish|Data Science| 10000|
|    Krish|         IOT|  5000|
|   Mahesh|    Big Data|  4000|
|    Krish|    Big Data|  4000|
|   Mahesh|Data Science|  3000|
|Sudhanshu|Data Science| 20000|
|Sudhanshu|         IOT| 10000|
|Sudhanshu|    Big Data|  5000|
|    Sunny|Data Science| 10000|
|    Sunny|    Big Data|  2000|
+---------+------------+------+



In [15]:
# Let's perform aggregation and filter like we did before
query = '''
select Departments, sum(salary) as highest_salary from employee_view
group by Departments
order by Departments desc;
'''
dept_salaries_summary = spark.sql(query)
dept_salaries_summary.show()

+------------+--------------+
| Departments|highest_salary|
+------------+--------------+
|         IOT|         15000|
|Data Science|         43000|
|    Big Data|         15000|
+------------+--------------+



### Interoperating the SQL and Dataframe API

In [16]:
krish_total_salary = spark.sql('select * from employee_view')\
.where("Name = 'Krish'")\
.groupBy('Name')\
.agg(sum('salary').alias('TotalSalary'))
krish_total_salary.show()

+-----+-----------+
| Name|TotalSalary|
+-----+-----------+
|Krish|      19000|
+-----+-----------+



### Writing data to a Parquet file

In [17]:
# Let's write a Dataframe to a parquet file
dept_salaries_summary.write.parquet("Data/department_salaries_summary.parquet", mode="overwrite")

In [18]:
# Let's write a Dataframe to a parquet file with Partition
dept_salaries_summary.write.parquet("Data/dept_salaries_with_part.parquet", partitionBy="Departments")

### Reading data from Parquet file

In [19]:
# Reading data from parquet file into a Dataframe
salaries_df = spark.read.parquet("data/department_salaries_summary.parquet")
salaries_df.show()

+------------+--------------+
| Departments|highest_salary|
+------------+--------------+
|         IOT|         15000|
|Data Science|         43000|
|    Big Data|         15000|
+------------+--------------+



In [22]:
# Reading data from a specified partition of a parquet file into a Dataframe
iot_dept_salaries = spark.read.parquet("Data/dept_salaries_with_part.parquet/Departments=Big Data")
iot_dept_salaries.show()

+--------------+
|highest_salary|
+--------------+
|         15000|
+--------------+



In [30]:
spark.sql("SELECT * FROM employeeParquetTable").show()

+------------+--------------+
| Departments|highest_salary|
+------------+--------------+
|         IOT|         15000|
|Data Science|         43000|
|    Big Data|         15000|
+------------+--------------+



### Convert Spark Dataframe to Pandas

In [35]:
# Convert the dataframe above to pandas dataframe
employee_df = spark.read.csv('Data/employees.csv', header=True, inferSchema=True)
employeePandasDf = employee_df.toPandas()
employeePandasDf.head()

,Name,Departments,salary
0,Krish,Data Science,10000
1,Krish,IOT,5000
2,Mahesh,Big Data,4000
3,Krish,Big Data,4000
4,Mahesh,Data Science,3000
